In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('/home/xmax/projects/nn_ansatz/src')

from nn_ansatz import *
from jax import vmap
import jax.numpy as jnp
from jax import vmap, lax
import numpy as np
import functools


/home/xmax/anaconda3/envs/drop/lib/python3.7/site-packages/pyscf/lib/misc.py:46: H5pyDeprecationWarning: Using default_file_mode other than 'r' is deprecated. Pass the mode to h5py.File() instead.
  h5py.get_config().default_file_mode = 'a'


In [23]:
from jax import random as rnd
from collections import OrderedDict

key = rnd.PRNGKey(1)
n_atom, n_up = 1, 1
in_lin, out_lin = 3, 10
n_det = 2
n_walkers = 2

walkers = rnd.normal(key, (n_walkers, 1, 3))
r_atoms = jnp.array([[0.0, 0.0, 0.0]])
params = OrderedDict()
params['linear'] = rnd.normal(key, (in_lin, out_lin))
params['env_linear'] = rnd.normal(key, (n_det, n_up, out_lin))
params['env_sigma'] = rnd.normal(key, (n_det, n_up, n_atom, 3, 3))
params['env_pi'] = rnd.normal(key, (n_det, n_up, n_atom))

def compute_ae_vectors_i(walkers: jnp.array,
                         r_atoms: jnp.array) -> jnp.array:
    r_atoms = jnp.expand_dims(r_atoms, axis=0)
    walkers = jnp.expand_dims(walkers, axis=1)
    ae_vectors = walkers - r_atoms
    return ae_vectors

def slogdet(x):
  if x.shape[-1] == 1:
    sign = jnp.sign(x[..., 0, 0])
    logdet = jnp.log(jnp.abs(x[..., 0, 0]))
  else:
    sign, logdet = jnp.linalg.slogdet(x)

  return sign, logdet

def logdet_matmul(xs) -> jnp.ndarray:
  dets = [x.reshape(*x.shape[:2], -1) for x in xs if x.shape[-1] == 1]
  dets = functools.reduce(
    lambda a, b: a*b, dets
  ) if len(dets) > 0 else 1.

  slogdets = [slogdet(x) for x in xs if x.shape[-1] > 1]
  maxlogdet = 0
  if len(slogdets) > 0:
    print('here')
    sign_in, logdet = functools.reduce(
      lambda a, b: (a[:, 0]*b[:, 0], a[:, 1]+b[:, 1]), slogdets
    )

    maxlogdet = jnp.max(logdet)
    det = sign_in * dets * jnp.exp(logdet - maxlogdet)
  else:
    det = dets
  print(det)
  
  result = jnp.sum(det, axis=1)

  sign_out = jnp.sign(result)
  log_out = jnp.log(jnp.abs(result)) + maxlogdet
  return log_out


def second_derivative(wf):

    def _lapl_over_f(params, walkers):
        walkers = walkers.reshape(n_walkers, -1)
        n = walkers.shape[-1]
        eye = jnp.eye(n, dtype=walkers.dtype)[None, ...].repeat(n_walkers, axis=0)
        grad_f = jax.grad(wf, argnums=1)
        grad_f_closure = lambda y: grad_f(params, y) 

        def _body_fun(i, val):

            print(walkers.shape, eye.shape, eye[..., i].shape)
            primal, tangent = jax.jvp(grad_f_closure, (walkers,), (eye[..., i],))
            print(primal.shape, tangent.shape, 'h')
            return val + primal[:, i]**2 + tangent[:, i]

        return -0.5 * lax.fori_loop(0, n, _body_fun, jnp.zeros((n_walkers,)))
    return _lapl_over_f


def create_wf(r_atoms):

    def _wf(params, walkers):

        if len(walkers.shape) == 2:  # this is a hack to get around the jvp
            walkers = walkers.reshape(n_walkers, n_up, 3)

        ae_vectors = walkers[:, None, ...] - r_atoms[None, None, ...] 

        linear = jnp.tanh(walkers @ params['linear'])
        
        print(linear.shape)
        factor = jnp.einsum('njf,kif->nkij', linear, params['env_linear'])

        argument = jnp.einsum('njmv,kimvc->njkimc', ae_vectors, params['env_sigma'])
        print(argument.shape)
        exponent = jnp.linalg.norm(argument, axis=-1)
        exponential = jnp.exp(-exponent)
        print(exponential.shape)
        orbitals = factor * jnp.einsum('njkim,kim->nkij', exponential, params['env_pi']) 
        print(orbitals.shape)

        orbitals *= 0.0

        log_psi = logdet_matmul([orbitals, orbitals])

        return log_psi.squeeze().sum()
    
    return _wf

wf = create_wf(r_atoms)
log_psi = wf(params, walkers)
print(log_psi.shape)
grad = second_derivative(wf)
print(log_psi)
grad(params, walkers)







(2, 1, 10)
(2, 1, 2, 1, 1, 3)
(2, 1, 2, 1, 1)
(2, 2, 1, 1)
[[[0.]
  [0.]]

 [[0.]
  [0.]]]
()
-inf
(2, 3) (2, 3, 3) (2, 3)
(2, 1, 10)
(2, 1, 2, 1, 1, 3)
(2, 1, 2, 1, 1)
(2, 2, 1, 1)
Traced<ShapedArray(float32[2,2,1])>with<JVPTrace(level=4/0)>
  with primal = Traced<ShapedArray(float32[2,2,1])>with<JVPTrace(level=2/0)>
                  with primal = Traced<ShapedArray(float32[2,2,1])>with<DynamicJaxprTrace(level=1/0)>
                       tangent = Traced<ShapedArray(float32[2,2,1])>with<DynamicJaxprTrace(level=1/0)>
       tangent = Traced<ShapedArray(float32[2,2,1]):JaxprTrace(level=3/0)>
(2, 3) (2, 3) h


DeviceArray([nan, nan], dtype=float32)

In [21]:
jnp.linalg.det(jnp.zeros((1,1)))


DeviceArray(0., dtype=float32)

In [8]:
import jax.numpy as jnp

def logabssumdet(orb_up: jnp.array,
                 orb_down: jnp.array) -> jnp.array:
    s_up, log_up = jnp.linalg.slogdet(orb_up)
    s_down, log_down = jnp.linalg.slogdet(orb_down)

    logdet_sum = log_up + log_down
    logdet_max = jnp.max(logdet_sum)

    argument = s_up * s_down * jnp.exp(logdet_sum - logdet_max)

    return jnp.log(jnp.abs(jnp.sum(argument, axis=0))) + logdet_max
orb_up = jnp.array([[0.]])[None, ...]
orb_down = jnp.array([[1., 2.], [-1., 2.]])[None, ...]

# logabssumdet(x, y)

s_up, log_up = jnp.linalg.slogdet(orb_up)
s_down, log_down = jnp.linalg.slogdet(orb_down)

logdet_sum = log_up + log_down
logdet_max = jnp.max(logdet_sum)

argument = s_up * s_down * jnp.exp(logdet_sum - logdet_max)

print(logdet_sum - logdet_max)

[nan]
